In [1]:
uv add pandas

Note: you may need to restart the kernel to use updated packages.


Resolved 22 packages in 1ms
Audited 21 packages in 0.04ms


In [2]:
uv add plotly

Note: you may need to restart the kernel to use updated packages.


Resolved 24 packages in 725ms
Prepared 2 packages in 10.40s
Installed 2 packages in 2.75s
 + narwhals==1.42.1
 + plotly==6.1.2


In [4]:
import pandas as pd

In [6]:
df=pd.read_pickle('processed_conversations2.pkl')

In [10]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go

# 1) Enable Plotly offline mode in Jupyter
init_notebook_mode(connected=True)

# 2) Reset your MultiIndex and sort by tweet order
df_reset  = df.reset_index()  # assumes df is indexed by ['airline','conversation','tweet']
df_sorted = df_reset.sort_values(['airline', 'conversation', 'tweet'])

# 3) Extract first sentiment_label and compute average sentiment_score from the 3rd tweet onward
first = (
    df_sorted[df_sorted['tweet'] == df_sorted.groupby(['airline','conversation'])['tweet'].transform('min')]
    .set_index(['airline','conversation'])['sentiment_label']
)
avg_scores = (
    df_sorted[df_sorted['tweet'] >= 2]
    .groupby(['airline','conversation'])['sentiment_score']
    .mean()
    .rename('avg_sentiment_score')
)

# 4) Combine and threshold back into labels
trans = (
    first.to_frame()
    .join(avg_scores, how='inner')
    .reset_index()
)

def score_to_label(s):
    if s < -0.4: return 'negative'
    if s <= 0.3: return 'neutral'
    return 'positive'

trans['avg_label'] = trans['avg_sentiment_score'].apply(score_to_label)

# 5) Count transitions from first → average
counts = trans.groupby(['sentiment_label', 'avg_label']).size().reset_index(name='count')

# 6) Define fixed node labels and colors
labels = [
    "Positive\n(first)", "Neutral\n(first)", "Negative\n(first)",
    "Positive\n(avg)",  "Neutral\n(avg)",  "Negative\n(avg)",
]
node_colors = ["#4CAF50", "#9E9E9E", "#F44336"] * 2
first_idx = {"positive": 0, "neutral": 1, "negative": 2}
avg_idx   = {"positive": 3, "neutral": 4, "negative": 5}

sources = counts['sentiment_label'].map(first_idx)
targets = counts['avg_label'].map(avg_idx)
values  = counts['count']
link_colors = [node_colors[src] for src in sources]

# 7) Set node positions: positive top (0.95), neutral middle (0.5), negative bottom (0.05)
x = [0, 0, 0,   1, 1, 1]
y = [0.95, 0.5, 0.05,   0.95, 0.5, 0.05]

# 8) Build the Sankey diagram
fig = go.Figure(go.Sankey(
    arrangement="fixed",
    node=dict(
        label=labels,
        color=node_colors,
        x=x, y=y,
        pad=20, thickness=30,
        line=dict(color="black", width=1),
        hovertemplate="%{label}<extra></extra>"
    ),
    link=dict(
        source=sources, target=targets, value=values,
        color=link_colors,
        hovertemplate="%{value} conversations<extra></extra>"
    )
))

# 9) Force a square layout and display
fig.update_layout(
    title_text="Sentiment Transition: First Tweet → Avg(3rd+) Tweet",
    width=600, height=600,
    font=dict(size=14, family="Arial"),
    margin=dict(l=50, r=50, t=80, b=50),
    plot_bgcolor="white"
)
iplot(fig)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed